방향키 : 이동
스페이스바 : 꾸욱~ 파워 조절, 때면 발사
벽은 맞으면 사라지고 3번 돌을 발사할때마다 벽 재생에너지 생김
외계인 발판 맞추면 밑으로 떨어짐
바닥 맞으면 바닥 파임

In [1]:
import math
import pygame
from pygame import draw
from pygame.color import Color
from pygame.sprite import Sprite
import time
from catapult import Catapult
from stone import Stone
from alien import Alien
from explosion import Explosion
from const import *
import copy
#from ground import Ground
import random
import pygame

class Ground():
    def __init__(self):
        self.ground=[[700,400],[0,400]]
        
        a=0
        for i in range(0,800,100):
            for j in range(0,100,10):
                a+=random.randint(-3,1)
                self.ground.append([i+j,a+300])
        

    def reshape(self,x):
        for i in range(5):
            self.ground[x+i][1]=max(self.ground[x][1]+10,self.ground[x+i][1])
        print('shape')
        return False

class Arc(pygame.sprite.Sprite):
    def __init__(self,x,y,w,h,alien):
        pygame.sprite.Sprite.__init__(self)
        self.screen=screen
        self.image=pygame.Surface((w,h))
        self.image.fill((0,255,0))
        self.rect=self.image.get_rect()
        self.rect.x=x
        self.rect.y=y
        self.w=w
        self.h=h
        self.alien=alien
        
    def draw(self,screen):
        pygame.draw.rect(screen,(0,255,0),(self.rect.x,self.rect.y,self.w,self.h))


#     def draw_wall(self,x):
#         pygame.draw.polygon(self.screen,(0,255,0),self.walls[x])
#         return alien_rect[x*4]
        
#     def draw_block(self,x):
#         pygame.draw.rect(self.screen,(0,255,0),self.block[x])
#         return alien_rect[x+1]

#         self.alien_rect=[350,465,460,300,660]
    
FPS = 60
stone_count = 30

def decrement_stones():
    global stone_count
    stone_count -= 1

class Background(Sprite):
    def __init__(self):        
        self.sprite_image = 'background.png'
        self.image = pygame.image.load(
                self.sprite_image).convert()
        self.rect = self.image.get_rect()
        self.rect.x = 0
        self.dx = 1

        Sprite.__init__(self)
        
    def update(self):
        self.rect.x -= self.dx
        if self.rect.x == -800:
            self.rect.x = 0

if __name__ == "__main__": 
    pygame.init()
    bomb=1
    size = (700, 300)
    screen = pygame.display.set_mode(size)
 
    pygame.display.set_caption("Catapult VS Alien")
 
    run = True
    clock = pygame.time.Clock()
    t = 0
    fire_sound = pygame.mixer.Sound('fire.ogg')
    crash_sound = pygame.mixer.Sound('crash.ogg')
    floor=0
    power = 15
    direction = 45
    slow=1
    game_state = GAME_INIT
    background = Background()
    background1 = Background()
    background1.rect.x =300
    background_group = pygame.sprite.Group()
    background_group.add(background)
    background_group.add(background1)
    
    stone = Stone() 
    stone.rect.y = -100   # 위치 변경
    stone_group = pygame.sprite.Group()
    stone_group.add(stone)
    
    stage =Ground()
#     self.wall1=[[300,300],[300,100],[320,100],[320,300]]
#     self.block=[[350,200,100,30],[460,200,100,30],[405,165,100,30]]
#     self.wall2=[[660,300],[660,150],[690,150],[690,300]]
#     self.alien_rect=[350,465,460,300,660]
    wall1=Arc(300,90,20,50,300)
    wall3=Arc(250,30,20,200,300)
    wall2=Arc(590,100,20,125,660)
    block1=Arc(350,50,100,30,350)
    block2=Arc(460,50,100,30,505)
    block3=Arc(405,15,100,30,460)
    
    block4=Arc(350,150,100,30,350)
    block5=Arc(460,150,100,30,505)
    castles=[wall1,wall3,block1,block2,block3,block4,block5,wall2]
    castle_group = pygame.sprite.Group()
    castle_group.add(castles)
    create=0
    
    fire=False
    
    catapult = Catapult(stone)    
    catapult.rect.x = 50 # 위치 변경                       
    catapult.rect.y = 400
    catapult_group = pygame.sprite.Group()
    catapult_group.add(catapult)
    a=[-5,-4,-3,-2,-1,0,1,2,3,4,5];
    w=0
    alien = Alien()
    alien.rect.x = 500
    alien.rect.y = 500-stage.ground[50][1]-32
    alien_temp_x=copy.deepcopy(alien.rect.x)
    #alien.Rotate(3)
    alien_group = pygame.sprite.Group()
    alien_group.add(alien)    
    high=0;
    pos=0
    broken=0;
    l=random.randint(-1,1)
    new=[[700,700],[710,710],[730,730]]
    position_slow=0
    explosion = Explosion()
    explosion_group = pygame.sprite.Group()
    explosion_group.add(explosion)
    # 게임 루프
    now= time.time()
    while run:
        castle_group.draw(screen)
        catapult_group.draw(screen)
        stone_group.draw(screen) 
        alien_group.draw(screen)
        pygame.draw.polygon(screen,(0,255,0),stage.ground)
        #pygame.draw.polygon(screen,(0,0,255),new)
        # 1) 사용자 입력 처리
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.KEYUP:
                
                if event.key == pygame.K_SPACE:
                    # 초기화면에서 스페이스를 입력하면 시작
                    if game_state == GAME_INIT: 
                        game_state = GAME_PLAY
                    elif game_state == GAME_PLAY: 
                        # GAME_PLAY 상태일 때 스페이스를 입력하면 발사
                        if stone.state == STONE_READY:
                            fire=True
                            stone_group.add(stone)
                            print('lunch')
                            t = 0
                            catapult.fire(power, direction)
                            fire_sound.play()
                elif event.key ==ord('a'):
                    bomb=1
                elif event.key ==ord('s'):
                    bomb=2
                elif event.key ==ord('d'):
                    bomb=3   

        if game_state == GAME_PLAY:
            # 누르고 있는 키 확인하기.
            keys = pygame.key.get_pressed()  
            if keys[pygame.K_LEFT]:
                catapult.backward()
            elif keys[pygame.K_RIGHT]:
                catapult.forward()
            elif keys[pygame.K_UP]:
                if direction < MAX_DIRECTION:
                    direction += 1
            elif keys[pygame.K_DOWN]:
                if direction > MIN_DIRECTION:
                    direction -= 1    
            elif keys[pygame.K_SPACE]:
                if power > MAX_POWER:
                    power = MIN_POWER
                else:
                    power += 0.2

        # 2) 게임 상태 업데이트
        if stone.state == STONE_FLY:
            t += 0.5
            stone.move(t, 
                       (screen.get_width(), screen.get_height()), 
                       decrement_stones)
#         if slow==1:
#             w=(w+1)%11
#         elif slow==0.5:
#             w=(w+1)%21
        #alien.rect.x=alien.rect.x+l*3*slow
        
        #---------------------------------------------------------
        if not castles in castle_group:
            broken=1
        else: broken=0
        print
        
        if alien.alive() and fire:
            
            for arc in castle_group:
                if pos==1 and arc==block4:
                    pos=0
                    floor=0
                elif pos==2 and arc==block5:
                    pos=0
                    floor=0
                
                if arc.rect.colliderect(stone.rect):
                    
                    fire=False
                    if high==0:
                        if not ((pos==1 and alien_temp_x<430) or(pos==2 and alien_temp_x>440)):
                            alien.rect.x = copy.deepcopy(alien_temp_x)
                    else :alien.rect.x = copy.deepcopy(alien_temp_x)
                    alien_temp_x=random.randint(320,550)
                        
#                     if arc== wall1:
#                         stone_group.remove(stone)
#                         break
                    create=create+1
                    if create>=3:
                        for i in range(8):
                            if castles[i] not in castle_group and create:
                                castle_group.add(castles[i])
                                create=0
                    castle_group.remove(arc)
                    stone_group.remove(stone)
                    
                    if arc==block4:
                        if 430>alien.rect.x>320:
                            high=1
                            floor=1
                        elif floor ==0 : pos=1
                    elif arc==block5 :
                        if alien.rect.x>450:
                            high=1
                            floor=1
                        elif floor ==0 : pos=2
                    break
            
                    
                
            collided = pygame.sprite.groupcollide(
                        stone_group, alien_group, False, True)
            if collided:
                explosion.rect.x = \
                    (alien.rect.x + alien.rect.width/2) - \
                     explosion.rect.width/2
                explosion.rect.y = \
                    (alien.rect.y + alien.rect.height/2) - \
                    explosion.rect.height/2
                crash_sound.play()
                
        
        elif not explosion.alive(): 
            # 외계인도 죽고 폭발 애니메이션도 끝났을 때.
            game_state = GAME_CLEAR
        pygame.draw.polygon(screen,(0,255,0),stage.ground)
        #pygame.draw.polygon(screen,(0,0,255),new)
        # 외계인이 살아 있는데 돌멩이 수가 0이면 게임 오버.
        if alien.alive() and stone_count == 0:
            game_state = GAME_OVER

        if game_state == GAME_PLAY: # 게임 객체 업데이트
            catapult_group.update()
            stone_group.update()
            alien_group.update()
        
        # 3) 게임 상태 그리기
        background_group.update()
        background_group.draw(screen)
        if game_state == GAME_INIT: 
            # 초기화면
            sf = pygame.font.SysFont("Arial", 20, bold=True)
            title_str = "Catapult VS Alien"
            title = sf.render(title_str, True, (255,0,0))
            title_size = sf.size(title_str)
            title_pos = (screen.get_width()/2 - title_size[0]/2, 100)
            
            sub_title_str = "Press [Space] Key To Start"
            sub_title = sf.render(sub_title_str, True, (255,0,0))
            sub_title_size = sf.size(sub_title_str)
            sub_title_pos = (screen.get_width()/2 - sub_title_size[0]/2, 200)

            screen.blit(title, title_pos)
            screen.blit(sub_title, sub_title_pos)

        elif game_state == GAME_PLAY: 
            # 플레이 화면
            c_x=int((catapult.rect.x+2-(catapult.rect.x+2)%10)/10)
            for i in range(4):
                while stage.ground[c_x+i][1]<catapult.rect.y+32:
                    catapult.rect.y-=1
                while stage.ground[c_x+i][1]>catapult.rect.y+34:
                    catapult.rect.y+=1
            c_x=int((alien.rect.x+2-(alien.rect.x+2)%10)/10)

            if high==0:
                alien.rect.y=118
            else :
                for i in range(4):
                    while stage.ground[c_x+i][1]<alien.rect.y+32:
                        alien.rect.y-=1
                    while stage.ground[c_x+i][1]>alien.rect.y+34:
                        alien.rect.y+=1
            catapult_group.draw(screen)
            stone_group.draw(screen) 
            alien_group.draw(screen)
            castle_group.draw(screen)
            pygame.draw.polygon(screen,(0,255,0),stage.ground) 
            #pygame.draw.polygon(screen,(0,0,255),new)
            # 파워와 각도를 선으로 표현.
            line_len = power*5 
            r = math.radians(direction)        
            pos1 = (catapult.rect.x+32, catapult.rect.y)
            pos2 = (pos1[0] + math.cos(r)*line_len, 
                    pos1[1] - math.sin(r)*line_len)
            draw.line(screen,Color(255, 0, 0), pos1, pos2)                 

            # 파워와 각도를 텍스트로 표현.
            sf = pygame.font.SysFont("Arial", 15)
            text = sf.render("{0} °, {1} m/s".
                             format(direction, int(power)), True, (0,0,0))
            screen.blit(text, pos2)

            # 돌의 개수를 표시
            sf = pygame.font.SysFont("Monospace", 20)
            text = sf.render("Stones : {0}".
                             format(stone_count), True, (0,0,255))
            screen.blit(text, (10, 10))

            if not alien.alive():
                explosion_group.update()
                explosion_group.draw(screen)

        elif game_state == GAME_CLEAR: 
            # 게임 클리어
            sf = pygame.font.SysFont("Arial", 20, bold=True)
            title_str = "Congratulations! Mission Complete"
            title = sf.render(title_str, True, (0,0,255))
            title_size = sf.size(title_str)
            title_pos = (screen.get_width()/2 - title_size[0]/2, 100)
            screen.blit(title, title_pos)

        elif game_state == GAME_OVER: 
            # 게임 오버
            sf = pygame.font.SysFont("Arial", 20, bold=True)
            title_str = "Game Over"
            title = sf.render(title_str, True, (255,0,0))
            title_size = sf.size(title_str)
            title_pos = (screen.get_width()/2 - title_size[0]/2, 100)
            screen.blit(title, title_pos)
            
        if fire:
            s_x=int((stone.rect.x+2-(stone.rect.x+2)%10)/10)

            if stone.rect.y>stage.ground[s_x][1]:
                print("hit")
                new=None
                new=[]
                new1=[]
                slow=None
                if abs(stage.ground[s_x][0]-alien.rect.x)<100:
                    a=[i-10 for i in range(21)];
                    slow=0.5
                else :
                    a=[-5,-4,-3,-2,-1,0,1,2,3,4,5];
                    slow=1
                
                for i in  range(6):
                    new.append(stage.ground[s_x+i])
                    
                new1.reverse()
                new=new+new1
                fire=stage.reshape(s_x)
                
                s_x=None
                stone.rect.y=0
                stone.rect.x=800
                pygame.draw.polygon(screen,(0,255,0),stage.ground)
                #pygame.draw.polygon(screen,(0,0,255),new)
                time.sleep(0.1)
                if high==0:
                    if not ((pos==1 and alien_temp_x<430) or(pos==2 and alien_temp_x>440)):
                        alien.rect.x = copy.deepcopy(alien_temp_x)
                else :
                    alien.rect.x = copy.deepcopy(alien_temp_x)
                alien_temp_x=random.randint(320,550)
                l=random.randint(-1,1)
                if broken==1:
                    create=create+1
                
        else:
            stone.rect.y=0
            stone.rect.x=750
            if create>=3:
                for i in range(8):
                    if castles[i] not in castle_group and create:
                        castle_group.add(castles[i])
                        create=0
        #if game_state == GAME_PLAY:
         #   pygame.draw.circle(screen,(255,0,0),[alien_temp_x,200],4,4)
            
        pygame.display.flip() 
        clock.tick(FPS)
        

    pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


<ipython-input-1-a0f5942ec4fe>:317: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  screen.blit(title, title_pos)
<ipython-input-1-a0f5942ec4fe>:318: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  screen.blit(sub_title, sub_title_pos)
<ipython-input-1-a0f5942ec4fe>:350: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  draw.line(screen,Color(255, 0, 0), pos1, pos2)
<ipython-input-1-a0f5942ec4fe>:356: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  screen.blit(text, pos2)


lunch
lunch
hit
shape
lunch
hit
shape
